<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DecisionTree</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

Сперва импортируем все необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Подготовка

Познакомимся с данными

In [2]:
#Создадим фрейм
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
#Проверим размерность
df.shape

(159571, 2)

In [4]:
#Проверим данные
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
#Проверим дисбаланс классов
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

У нас текст на английском языке. Начнем работу по подготовке данных:

1) Проведем токенизацию

2) Проведем стемминг

3) Очистим текст от стоп слов

4) Разделим выбрки на обучающую и тестовую

5) Для векторизации будем использовать TF-IDF

6) При обучении моеделей стоит учитывать дисбаланс классов.

Начнем нашу работу.

In [6]:
#Разделим данные на целевой и обучающий признаки
y = df['toxic']
X = df['text']

In [7]:
#Создадим стеммер и стоп-слова
snowball = SnowballStemmer(language="english")
stop_words = stopwords.words("english")

In [8]:
#Напишем функцию для предобработки текста
def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    #токенизируем фразы
    tokens = word_tokenize(sentence, language="english")
    
    #уберем знаки препинания
    tokens = [i for i in tokens if i not in string.punctuation]
    
    #укажем действие при необходимости удаления стоп-слов
    if remove_stop_words:
        tokens = [i for i in tokens if i not in stop_words]
        
    #проведем стемминг
    tokens = [snowball.stem(i) for i in tokens]
    
    return tokens

In [9]:
#разобьем выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.25, random_state = 1123, stratify=y)

In [10]:
#Создадим векторный преобразователь с функцией токенизации
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [11]:
#произведем обучение с преобразованием для тренировочной выборки, а тестовую просто преобразуем для избежания утечки
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## Обучение

Для дальнейшей работы я напишк 2 функции которые позволят автоматизировать вычисления:

1) будет высчитывать необходимую метрику

2) будет обучать модель и возвращать метрики на тренировочной и тестовой выборках

In [12]:
def f1_metric(model, X, y):

    metric = 0
    
    y_pred = model.predict(X)
    
    metric = metrics.f1_score(y, y_pred)
    
    return metric


In [13]:
def ev_model(model, X_train, y_train, X_test, y_test, scale=False):

    metric_train = 0
    metric_test = 0
    
    if (scale == True):
      
      sc = StandardScaler()

      X_train = sc.fit_transform(X_train)
      
      X_test = sc.transform(X_test)
    
    model.fit(X_train, y_train)
  
    metric_train = f1_metric(model, X_train, y_train)

    metric_test = f1_metric(model, X_test, y_test)
    
    return metric_train, metric_test


### LogisticRegression

In [14]:
#Do parameters of gridsearch 
param_grid = [{'penalty' : ['l1', 'l2'], 'C' : [10**i for i in range(-20,7)]}]


#Do gridsearch
scv = StratifiedKFold(n_splits=5)

LR=GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid=param_grid, scoring='f1', cv=scv, verbose=True, n_jobs=-1)

best_LR= LR.fit(X_train, y_train)


print('best parameters (CV f1 score =%0.3f):' % best_LR.best_score_)

print("best model:\n", best_LR.best_params_)

In [15]:
#Воспользуемся нашими функциями и проверим какие показатели удалось получить
LR_report=ev_model(LogisticRegression(penalty='l1', C=10, solver = 'liblinear', class_weight='balanced'), X_train, y_train, X_test, y_test, scale=False)

print('f1-score of the best model on train sample = %0.3f:' %LR_report[0])
print('f1-score of the best model on test sample = %0.3f:' %LR_report[1] )

f1-score of the best model on train sample = 0.954:
f1-score of the best model on test sample = 0.764:


Модель переобучилась, однако нам удалось пройти бейзлайн

### DecisionTree

In [16]:
#Do parameters of gridsearch 
param_grid = [{ 
    'max_depth' : list(range(2,4)),
    'min_samples_leaf' : list(range(2,4)), 
    'min_samples_split' : list(range(2,4))
     }]


#Do gridsearch


DT=GridSearchCV(DecisionTreeClassifier(class_weight='balanced'), param_grid=param_grid, scoring='f1', verbose=True, n_jobs=-1)

best_DT=DT.fit(X_train, y_train)



print('best parameters (CV f1 score =%0.3f):' % best_DT.best_score_)

print("best model:\n", best_DT.best_params_)

In [17]:
DT_report=ev_model(DecisionTreeClassifier(class_weight='balanced', max_depth = 20, min_samples_split = 8, min_samples_leaf = 4), X_train, y_train, X_test, y_test, scale=False)

print('f1-score of the best model on train sample = %0.3f:' %DT_report[0])
print('f1-score of the best model on test sample = %0.3f:' %DT_report[1])

f1-score of the best model on train sample = 0.671:
f1-score of the best model on test sample = 0.627:


Бейзлайн не пройден

## Выводы

Для выполнения данной работы мы использовали стемминг и TFIDF для векторизации текстов. А так же обучили 2 модели: Логистическую регрессию и DecisionTree. Лучше себя показала логистическая регрессия (0.764 против 0.627 на тестовой выборке), нам удалось преодолеть бэйзлайн.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны